# Halftone Code and Exercises

In this notebook we create a function that mimics the effect of halftone printing on digital images. Much like the original photomechanical process, the algorithm creates larger black dots when the same area in the original image is darker. It does this by dividing the entire image into small areas and then adding up the darkness values or "ink" from the pixels within each these areas. The resulting number is used to determine the size of a simulated ink dot (a black circle) at that position in the resulting image.

In order to simulate a wide range of light and dark halftones the resolution of the output image must be significantly larger than the original image resolution. This is because you need to be able to render a range of ink dot sizes, some dots taking up many more pixels than other dots. So each dot area requires a large region in the resulting image, since some dots only use a few pixels, but others large dots will fill most of the region.

## Settings
There are two important settings that shape the way this algorithm works. The scaling factor (scale) setting is a positive decimal value which will increase the size of the output halftone image by a multiple of the original image resolution, so a scaling factor of two will create a halftone image that is twice the original image size. A higher scaling factor increases output resolution, which allows for a wider range of "ink dot" sizes. Scale is also used to simulate different printing contexts. For instance, a higher scaling factor might simulate that a photo was printed on most of a newspaper page, such as a front page image. Smaller scale might simulate an image printed within a column of text, as in an advertisement or an obituary photo.

The sampling distance (sample dist) setting is a length, given in pixels. All of the pixels that are within that sampling distance of an ink dot location on the original, not yet scaled up, image are added up to determine the ink dot size. So sampling distance determines the region of pixels that are "sampled" by the ink dot size calculation.

## Code
The code block below defines the Python function for rendering a halftone image from an input image, given a sampling distance and a scaling factor. We will use this function in later demonstrations.

In [1]:
from PIL import Image, ImageDraw, ImageStat

# Adapted from a Stack Overflow answer: https://stackoverflow.com/a/47834501/1600958
def halftone(img, sample=8, scale=1, angle=45):
    ''' Returns a halftone image created from the given input image `img`.
    `sample` (in pixels), determines the sample box size from the original
    image. The maximum output dot diameter is given by `sample` * `scale`
    (which is also the number of possible dot sizes). So `sample` == 1 will
    preserve the original image resolution, but `scale` must be > 1 to allow
    variations in dot size.
    '''
    img_grey = img.convert('L')  # Convert to greyscale.
    channel = img_grey.split()[0]  # Get grey pixels.
    channel = channel.rotate(angle, expand=1)
    size = channel.size[0]*scale, channel.size[1]*scale

    bitmap = Image.new('1', size)
    draw = ImageDraw.Draw(bitmap)

    for x in range(0, channel.size[0], sample):
        for y in range(0, channel.size[1], sample):
            box = channel.crop((x, y, x+sample, y+sample))
            mean = ImageStat.Stat(box).mean[0]
            diameter = (mean/255) ** 0.5
            edge = 0.5 * (1-diameter)
            x_pos, y_pos = (x+edge) * scale, (y+edge) * scale
            box_edge = sample * diameter * scale
            draw.ellipse((x_pos, y_pos, x_pos+box_edge, y_pos+box_edge),
                         fill=255)

    bitmap = bitmap.rotate(-angle, expand=1)
    width_half, height_half = bitmap.size
    xx = (width_half - img.size[0]*scale) / 2
    yy = (height_half - img.size[1]*scale) / 2
    bitmap = bitmap.crop((xx, yy, xx + img.size[0]*scale,
                                  yy + img.size[1]*scale))
    return Image.merge('1', [bitmap])



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Student Exercise: Compare a Scanned Photo with Halftone

Let's take the vibrant grayscale image of Michael Jordan making a slam dunk and see how it looks when rendered as a halftone image. We will start out with some high quality default settings for the scale factor and sampling distance. These settings would require a large print area on a newspaper page. After you run the code block below, wait a few seconds for the images to appear, then examine the halftone effect with the default settings. You can also see how big this image would appear on a newspaper page. Note that the quality of the Michael Jordan image overlaid on the newspaper is not representative of halftone, but it gives you a sense of the image's print size. (That newspaper rendering cannot be representative because the simulated ink dots are highly distorted when scaled down smaller than the pixels available on the screen.)

Standard US newspaper pages are 15 inches wide and they were normally capable of printing halftone images at 80 dots per inch. Try adjusting the sliders to see the effect of different settings. The image will update a few seconds after you change a setting.

To render the image in a single newspaper column, trying a scale factor of 12 and a sampling distance of 5.
To render it as a front page image, try a scale factor of 15 and a sampling distance of 1.

In [2]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # We are dealing with a large image from a known source
from io import BytesIO
from ipywidgets import Image as Image2
from ipywidgets import widgets, interact, interactive, fixed, interact_manual

box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')

vbox_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='75%')

newspaper_im = Image.open('images/Afro-American_1928.png')
def newsprint_overlay(image, dotwidth, zoom=2):
    newspaper = newspaper_im.resize((newspaper_im.size[0]*zoom, newspaper_im.size[1]*zoom))
    image_size = image.size
    new_image = Image.new('RGB',(newspaper.size[0], newspaper.size[1]), (250,250,250))
    new_image.paste(newspaper,(0,0))
    # newspaper is 15in wide x 80 dpi, = X dots wide
    np_dotwidth = 15*80
    im_proportion = dotwidth/np_dotwidth
    scaled_image = image.resize((int(new_image.size[0]*im_proportion), int(new_image.size[1]*im_proportion)))
    new_image.paste(scaled_image,(int(newspaper.size[0]/2),int(newspaper.size[1]/2)))
    return new_image

def htw(im, scale, sample):
    original_out = widgets.Output(layout={'width': '50%', 'border': '1px solid black'})
    halftone_out = widgets.Output(layout={'width': '50%', 'border': '1px solid black'})
    original_full_out = widgets.Output(layout={'width': '50%', 'height': '50px', 'border': '1px solid black'})
    halftone_full_out = widgets.Output(layout={'width': '50%', 'border': '1px solid black'})
    newsprint_out = widgets.Output(layout={'width': '100%', 'border': '1px solid black'})
    with original_out:
        print('Input Image (resized to fit page)')
        display(im)
    ht = halftone(im, scale=scale, sample=sample)
    with halftone_out:
        print('Simulated Halftone (resized to fit page)')
        display(ht)
    with original_full_out:
        print('Original image at full resolution')
        b = BytesIO()
        crop = im.crop((100, 200, 300, 400))
        crop = crop.resize((crop.size[0]*scale, crop.size[1]*scale))
        display(crop)
    with halftone_full_out:
        print('Halftone image at full resolution')
        b = BytesIO()
        crop = ht.crop((100*scale, 200*scale, 300*scale, 400*scale))
        display(crop)
    with newsprint_out:
        im_dotwidth = int(ht.size[0]/(sample*scale))
        print('Newsprint Context: Halftone image is %s ink dots wide. Michael Jordan image below is not halftone, but shows print size.'%(im_dotwidth))
        display(newsprint_overlay(im, im_dotwidth))
    top_box = widgets.HBox([original_out, halftone_out], layout=box_layout)
    mid_box = widgets.HBox([original_full_out, halftone_full_out], layout=box_layout)
    box = widgets.VBox([top_box, mid_box, newsprint_out], layout=vbox_layout)
    display(box)
    
    
original = Image.open('images/Jordan.png')
one = interactive(htw, im=fixed(original), 
                  scale=widgets.IntSlider(description='Scale Factor:',min=1, max=15, step=1, value=15), 
                  sample=widgets.IntSlider(description='Sample Dist:', min=1, max=10, step=1, value=2))
display(one)

ModuleNotFoundError: No module named 'ipywidgets'

## Next Step

Next explore a simulation of the microfilm archiving photochemical process. Open file [microfilm.ipynb](microfilm.ipynb) to proceed.